In [3]:
import torch
from torch import nn
import numpy as np
import os
import time
import codecs
import re
import json
import pandas as pd
import glob

In [4]:
#get weight data
rootdir = "MyFitbitData5/Mad/personal & Account"
#rootdir = "MyFitbitData/name/personal & Account"
regex = re.compile('weight-2021-03-05')
#regex = re.compile('weight-2021-most-recent')

filenames = os.listdir(rootdir)
weights= []
for root, dirs, files in os.walk(rootdir):
    for file in files:
        if regex.match(file):
            weights = np.append(weights, file)

print(weights)            
for x in range(np.size(weights)):
    filename = rootdir + "/" + weights[x]    
    data = []
    weight = []
    fat = []
    with open(filename) as json_file:
        roughdata = json.load(json_file)
        for x in range(np.size(roughdata)):
            del roughdata[x]["logId"]
            del roughdata[x]["time"]
            del roughdata[x]["source"]
            del roughdata[x]["bmi"]
            weight = np.append(weight, roughdata[x]["weight"])
            fat = np.append(fat, roughdata[x]["fat"])

        data = np.append(data, roughdata)
weightfat = data
print(weightfat)
weeklyWeights = [(weightfat[0]["weight"] + weightfat[1]["weight"])/2]
weeklyfats = [(weightfat[0]["fat"] + weightfat[1]["fat"])/2]
      
for x in range(len(weightfat)):
    if (x+1) % 7 == 0:
        if x + 1 == len(weightfat):
            weeklyWeights = np.append(weeklyWeights, (weightfat[x]["weight"] + weightfat[x-1]["weight"] )/2)
            weeklyfats = np.append(weeklyfats, (weightfat[x]["fat"] + weightfat[x-1]["fat"])/2)

        else:
            weeklyWeights = np.append(weeklyWeights, (weightfat[x]["weight"] + weightfat[x+1]["weight"] + weightfat[x-1]["weight"])/3)
            weeklyfats = np.append(weeklyfats, (weightfat[x]["fat"] + weightfat[x+1]["fat"] + weightfat[x-1]["fat"])/3)
print(weeklyWeights)
print(weeklyfats)
        
        
        

['weight-2021-03-05.json']
[{'weight': 116.6, 'fat': 16.0, 'date': '03/11/21'}
 {'weight': 116.6, 'fat': 16.0, 'date': '03/12/21'}
 {'weight': 114.6, 'fat': 15.300000190734863, 'date': '03/13/21'}
 {'weight': 114.6, 'fat': 14.5, 'date': '03/14/21'}
 {'weight': 112.0, 'fat': 13.899999618530273, 'date': '03/15/21'}
 {'weight': 112.8, 'fat': 13.199999809265137, 'date': '03/16/21'}
 {'weight': 112.8, 'fat': 13.199999809265137, 'date': '03/17/21'}
 {'weight': 112.0, 'fat': 14.600000381469727, 'date': '03/18/21'}
 {'weight': 112.2, 'fat': 14.600000381469727, 'date': '03/19/21'}
 {'weight': 110.0, 'fat': 14.600000381469727, 'date': '03/20/21'}
 {'weight': 110.8, 'fat': 13.899999618530273, 'date': '03/21/21'}
 {'weight': 109.8, 'fat': 13.899999618530273, 'date': '03/22/21'}
 {'weight': 109.8, 'fat': 13.899999618530273, 'date': '03/23/21'}
 {'weight': 110.4, 'fat': 14.600000381469727, 'date': '03/24/21'}
 {'weight': 107.6, 'fat': 14.600000381469727, 'date': '03/25/21'}
 {'weight': 109.8, 'fat':

In [5]:
#get calorie data
rootdir = "MyFitbitData5/Mad/Nutrition"

regex = re.compile('food_logs-0')
filenames = os.listdir(rootdir)
foodNames = []
for root, dirs, files in os.walk(rootdir):
    for file in files:
        if regex.match(file):
            foodNames = np.append(foodNames, file)

start = False
#127
for x in range(np.size(foodNames)):
    filename = 'MyFitbitData5/Mad/Nutrition/' + foodNames[x]
    data = []
    count = 0
    print(filename)
    with open(filename) as json_file:
        roughdata = json.load(json_file)
        
        for y in range(np.size(roughdata)):
            
            if roughdata[y-count]["logDate"] == "2021-03-27":
                start = True
            if roughdata[y-count]["logDate"] == "2021-03-10":
                start = False
            
            if start == True:
                cal = roughdata[y-count]["loggedFood"]["calories"]
                del roughdata[y-count]["logId"]
                del roughdata[y-count]["loggedFood"]
                del roughdata[y-count]["favorite"]
                if "nutritionalValues" in roughdata[y-count].keys():                
                    del roughdata[y-count]["nutritionalValues"]                
                roughdata[y-count]["calories"] = cal
            else:
                del roughdata[y-count]
                count +=1

                
        
        data = np.append(data, roughdata)
calories = data[0]["calories"]
totCalories = np.empty((0,1))
dates = np.empty((0,1))
dates = np.vstack((dates, [data[0]["logDate"]]))
for x in range(len(data)-1):
    if data[x+1]["logDate"] == data[x]["logDate"]:
        calories += data[x+1]["calories"]
        if x == len(data)-2:
            totCalories = np.vstack((totCalories, calories))
    else:
        dates = np.vstack((dates, data[x+1]["logDate"]))
        totCalories = np.vstack((totCalories, calories))
        print(data[x+1])
        calories = data[x+1]["calories"]
        
    

totCalories = np.flip(np.vstack((totCalories, [[1000]])))
print(totCalories)

weeklyTotCalories = []
for x in range(len(totCalories)):
    if (x+1) % 7 == 0:
        weeklyTotCalories = np.append(weeklyTotCalories, np.sum(np.transpose(totCalories)[0][x-6:x+1]))
    

print(weeklyTotCalories)

MyFitbitData5/Mad/Nutrition/food_logs-0.json
{'logDate': '2021-03-26', 'calories': 140}
{'logDate': '2021-03-25', 'calories': 211}
{'logDate': '2021-03-24', 'calories': 177}
{'logDate': '2021-03-23', 'calories': 283}
{'logDate': '2021-03-22', 'calories': 167}
{'logDate': '2021-03-21', 'calories': 80}
{'logDate': '2021-03-20', 'calories': 226}
{'logDate': '2021-03-19', 'calories': 409}
{'logDate': '2021-03-18', 'calories': 275}
{'logDate': '2021-03-17', 'calories': 252}
{'logDate': '2021-03-16', 'calories': 342}
{'logDate': '2021-03-15', 'calories': 180}
{'logDate': '2021-03-14', 'calories': 340}
{'logDate': '2021-03-13', 'calories': 423}
{'logDate': '2021-03-12', 'calories': 266}
[[1000.]
 [ 567.]
 [1052.]
 [ 932.]
 [ 601.]
 [ 810.]
 [ 775.]
 [ 757.]
 [1021.]
 [2191.]
 [ 658.]
 [ 744.]
 [1097.]
 [ 632.]
 [ 857.]
 [1611.]
 [1261.]]
[5737. 7100.]


In [6]:
def caloriesBurned(weight, fat):
    fatMass = []
    leanMass = []
    for x in range(len(weight)):
        fatMass = np.append(fatMass, weight[x]*fat[x]/100)
        leanMass = np.append(leanMass, weight[x] - fatMass[x])
    calories = [] 
    print(fatMass)
    print(leanMass)
    for x in range(len(weight) - 1):   
        calories = np.append(calories, -(fatMass[x]-fatMass[x+1])*4423.9)
        if leanMass[x] >= leanMass[x+1]:
            calories[x] -= (leanMass[x] - leanMass[x+1])*700
        else:
            calories[x] -= (leanMass[x]-leanMass[x+1])*2650
    #calories = np.append(calories, [0])

    return calories
        
    
calories = caloriesBurned(weeklyWeights, weeklyfats)
print(calories)



def weeklyCaloriesBurned(weight, fat):
    fatMass = []
    leanMass = []
    print(len(weight))
    for x in range(len(weight)):
        fatMass = np.append(fatMass, weight[x]*fat[x]/100)
        leanMass = np.append(leanMass, weight[x] - fatMass[x])
    calories = [] 
    print(fatMass)
    for x in range(len(weight) - 1):   
        calories = np.append(calories, -(fatMass[x]-fatMass[x+1])*4423.9)
        if leanMass[x] >= leanMass[x+1]:
            calories[x] -= (leanMass[x] - leanMass[x+1])*700
        else:
            calories[x] -= (leanMass[x]-leanMass[x+1])*2650

    return calories
        

weeklyCalories = []
for x in range(len(weeklyTotCalories)):
    
   # weeklyCalories = np.append(weeklyCalories, weeklyTotCalories[x] + np.sum(calories[x*7:(x*7) + 7]))
    weeklyCalories = np.append(weeklyCalories, weeklyTotCalories[x] - calories[x])
    
#calories = np.append(calories, [2000,2000, 2000])
print(weeklyCalories)

[18.656      15.37955556 15.69797792]
[97.944      97.15377778 93.56868875]
[-15047.81813333  -1100.89363594]
[20784.81813333  8200.89363594]


In [15]:
#get heart rates
rootdir = "MyFitbitData5/Mad/Physical Activity"
#rootdir = "MyFitbitData/Name/Physical Activity"
regex = re.compile('heart_rate-2021-03')
filenames = os.listdir(rootdir)
heartRates = []
for root, dirs, files in os.walk(rootdir):
    for file in files:
        if regex.match(file):
            heartRates = np.append(heartRates, file)


#read files for hr data
inputs = np.empty((0,3))
dateTime = []
start = False
minInputs = np.empty((0,3))
hrDates = np.empty((0,1))
count = 1
for x in range(np.size(heartRates)):
    filename = 'MyFitbitData5/Mad/Physical Activity/' + heartRates[x]
    data = []
    
    with open(filename) as json_file:
        data = json.load(json_file)
        inputs = np.empty((len(data), 3))
        for y in range(len(data)):
            temp = data[y]["dateTime"][9:] 
            
            
            if data[y]["dateTime"][:8] == "03/11/21":
                start = True
                
            if data[y]["dateTime"][:8] == "03/28/21":
                start = False
            if start == True:
                inputs[y][0] = int(temp[:2])*3600  + int(temp[3:5])*60 + int(temp[6:9])
                inputs[y][1] = int(data[y]["value"]["bpm"])
                inputs[y][2] = int(count/7)
            
       # hrDates = np.vstack((hrDates, data[10]["dateTime"][:8]))
        if start == True:
            count += 1
            minInputs = np.vstack((minInputs, inputs))



from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1,1))

for x in range(len(minInputs) - 1):
    if minInputs[x][0] <= minInputs [x+1][0]:
        minInputs[x][0] = minInputs[x+1][0] - minInputs[x][0]
    else:
        minInputs[x][0] = 86400 - minInputs[x][0] + minInputs[x+1][0]
        
        
print(minInputs)    
'''if minInputs[x][0] > 10:
        tot = int(minInputs[x][0]/10)
        print(tot)
        print(minInputs[x])
        
        minInputs[x][0] = minInputs[x][0]%10
        
        for y in range(tot):    
            intervals = minInputs[0:x+y, 0:1]
            currentInterval = np.sum(intervals)           
            minInputs = np.insert(minInputs, [x+1],[currentInterval, (minInputs[x][1]+minInputs[x+1][1])/2,int((currentInterval+((tot-y)*10))/86400)], axis=0)'''
weeklyInputs = minInputs[:144547]
minInputs = weeklyInputs
train_data_normalized2 = scaler.fit_transform(np.transpose(minInputs)[1] .reshape(-1,1))
scaler = MinMaxScaler(feature_range=(-1,1))

train_data_normalized1 = scaler.fit_transform(np.transpose(minInputs)[0] .reshape(-1,1))

scaler = MinMaxScaler(feature_range=(-1,1))

train_data_normalized3 = scaler.fit_transform(np.transpose(minInputs)[2] .reshape(-1,1))

train_data_normalized = np.hstack((train_data_normalized1, train_data_normalized2))

train_data_normalized = np.hstack((train_data_normalized, train_data_normalized3))
train_data_normalized =torch.FloatTensor(train_data_normalized)
inputs = train_data_normalized.view(-1, 11119, train_data_normalized.shape[1])
print(inputs)
numOfInputs = np.size(np.transpose(minInputs)[0])

[[5.0000e+00 4.2000e+01 0.0000e+00]
 [1.5000e+01 4.3000e+01 0.0000e+00]
 [5.0000e+00 4.2000e+01 0.0000e+00]
 ...
 [5.0000e+00 5.1000e+01 2.0000e+00]
 [5.0000e+00 5.0000e+01 2.0000e+00]
 [8.6389e+04 5.1000e+01 2.0000e+00]]
tensor([[[-0.9977, -0.8977, -1.0000],
         [-0.9919, -0.8864, -1.0000],
         [-0.9977, -0.8977, -1.0000],
         ...,
         [-0.9948, -0.8750, -1.0000],
         [-0.9919, -0.8864, -1.0000],
         [-0.9977, -0.8977, -1.0000]],

        [[-0.9919, -0.9091, -1.0000],
         [-0.9977, -0.9091, -1.0000],
         [-0.9919, -0.8864, -1.0000],
         ...,
         [-0.9977, -0.8750, -1.0000],
         [-0.9977, -0.8977, -1.0000],
         [-0.9977, -0.9205, -1.0000]],

        [[-0.9977, -0.9432, -1.0000],
         [-0.9977, -0.9545, -1.0000],
         [-0.9919, -0.9659, -1.0000],
         ...,
         [-0.9977, -0.9318, -1.0000],
         [-0.9919, -0.9432, -1.0000],
         [-1.0000, -0.9432, -1.0000]],

        ...,

        [[-0.9977, -0.7614,  1.0

In [42]:
y_actual = []
week = 0
for x in range(len(weeklyInputs)):
    interval = weeklyInputs[x][0]
    day = weeklyInputs[x][2]
   
    intervalCalorie = weeklyCalories[int(day)]/(86400 *(day+1) * 7) * interval        
    y_actual = np.append(y_actual, intervalCalorie)

print(y_actual)
yscaler = MinMaxScaler(feature_range=(-1,1))

normalized_y = torch.FloatTensor(yscaler.fit_transform(y_actual .reshape(-1,1)))
outputs = normalized_y.view(-1, 11119, normalized_y.shape[1])
print(normalized_y)
print(outputs)

[0.17183216 0.51549648 0.17183216 ... 0.06779839 0.06779839 0.10169759]
tensor([[-0.9972],
        [-0.9914],
        [-0.9972],
        ...,
        [-0.9990],
        [-0.9990],
        [-0.9984]])
tensor([[[-0.9972],
         [-0.9914],
         [-0.9972],
         ...,
         [-0.9943],
         [-0.9914],
         [-0.9972]],

        [[-0.9914],
         [-0.9972],
         [-0.9914],
         ...,
         [-0.9972],
         [-0.9972],
         [-0.9972]],

        [[-0.9972],
         [-0.9972],
         [-0.9914],
         ...,
         [-0.9972],
         [-0.9914],
         [-0.9995]],

        ...,

        [[-0.9995],
         [-0.9990],
         [-0.9995],
         ...,
         [-0.9995],
         [-0.9990],
         [-0.9995]],

        [[-0.9994],
         [-0.9995],
         [-0.9990],
         ...,
         [-0.9995],
         [-0.9995],
         [-0.9984]],

        [[-0.9984],
         [-0.9984],
         [-0.9984],
         ...,
         [-0.9990],
         [-0

In [45]:
class RNN(nn.Module):
    def __init__(self, input_size = 3, hidden_size = 30, output_size = 1):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size, hidden_size, 1)
        self.linear = nn.Linear(hidden_size, output_size)
        
        self.hidden_cell = (torch.zeros(1,1,self.hidden_size),torch.zeros(1,1,self.hidden_size))
        
        
    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq),1,-1), self.hidden_cell)
        predictions = self.linear(lstm_out)
        return predictions
    
n_hidden = 5
lstm = RNN()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.05)
print(lstm)

RNN(
  (lstm): LSTM(3, 30)
  (linear): Linear(in_features=30, out_features=1, bias=True)
)


In [46]:
epochs = 100


for i in range(epochs):
    for x in range(13):    
        optimizer.zero_grad()
        lstm.hidden_cell = (torch.zeros(1,1, lstm.hidden_size), torch.zeros(1,1,lstm.hidden_size))
        y_pred = lstm(inputs[x])
        single_loss = loss_function(y_pred.float(), outputs[x].view(11119, 1, -1).float())
        single_loss.backward()
        optimizer.step()
    
    
    
    print(f'epoch: {i:3} loss:', single_loss.item())

epoch:   0 loss: 0.003593391738831997
epoch:   1 loss: 0.017247114330530167
epoch:   2 loss: 0.003009571461006999
epoch:   3 loss: 0.005372293293476105
epoch:   4 loss: 0.005323371849954128
epoch:   5 loss: 0.0011006661225110292
epoch:   6 loss: 0.0009792943019419909
epoch:   7 loss: 5.059611066826619e-05
epoch:   8 loss: 0.00011365759564796463
epoch:   9 loss: 4.304902631702134e-06
epoch:  10 loss: 1.9174813132849522e-05
epoch:  11 loss: 1.2216298273415305e-05
epoch:  12 loss: 2.4886323899409035e-06
epoch:  13 loss: 6.6234447331225965e-06
epoch:  14 loss: 6.048451723472681e-06
epoch:  15 loss: 1.8299780322195147e-06
epoch:  16 loss: 1.38804361995426e-05
epoch:  17 loss: 1.0941384061879944e-05
epoch:  18 loss: 1.85488636361697e-06
epoch:  19 loss: 2.5050117983482778e-05
epoch:  20 loss: 1.973269354493823e-05
epoch:  21 loss: 1.9447791146376403e-06
epoch:  22 loss: 4.239390545990318e-05
epoch:  23 loss: 4.1960811358876526e-05
epoch:  24 loss: 1.2469529337977292e-06
epoch:  25 loss: 3.12

In [71]:
num = 0
y_pred = lstm(inputs[num])
single_loss = loss_function(y_pred.float(), outputs[num].view(11119, 1, -1).float())
yo = y_pred.detach().numpy()
print(single_loss.item())
woo = yscaler.inverse_transform(np.transpose(np.transpose(yo)[0]))
print(np.sum(woo))
#print(woo)

outs = yscaler.inverse_transform(outputs[num])
print(np.sum(outs))
print(np.hstack((woo,outs)))


7.111459581210511e-06
2822.8765
2834.7923764862917
[[0.38212571 0.17183342]
 [0.29184961 0.51549702]
 [0.32912183 0.17183342]
 ...
 [0.31014141 0.34366345]
 [0.3114621  0.51549702]
 [0.31944269 0.17183342]]
